In [ ]:
import pandas as pd

# Load the main abundance table
data = pd.read_excel('../data/abundance_tables.xlsx', sheet_name=None)
# Check structure
for sheet_name, df in data.items():
    print(f"Sheet: {sheet_name}, Shape: {df.shape}")

Sheet: phylum, Shape: (52, 170)
Index(['phylum', 'normalized abundance (no unassigned)', 'mean abundance',
       'mean cv', 'mean subjects', 'found in datasets', 'mean abundance D01',
       'mean abundance D02', 'mean abundance D03', 'mean abundance D04',
       ...
       'D09_S07', 'D09_S08', 'D09_S09', 'D09_S10', 'D10_S01', 'D10_S02',
       'D10_S03', 'D10_S04', 'D10_S05', 'D10_S06'],
      dtype='object', length=170)
Sheet: genus, Shape: (422, 170)
Index(['genus', 'normalized abundance (no unassigned)', 'mean abundance',
       'mean cv', 'mean subjects', 'found in datasets', 'mean abundance D01',
       'mean abundance D02', 'mean abundance D03', 'mean abundance D04',
       ...
       'D09_S07', 'D09_S08', 'D09_S09', 'D09_S10', 'D10_S01', 'D10_S02',
       'D10_S03', 'D10_S04', 'D10_S05', 'D10_S06'],
      dtype='object', length=170)
Sheet: KO, Shape: (2758, 172)
Index(['KO', 'KO name', 'pathways', 'normalized abundance (no unassigned)',
       'mean abundance', 'mean cv', 'me

In [18]:
xls = pd.ExcelFile("../data/abundance_tables.xlsx")
phylum = pd.read_excel(xls, "phylum")
genus = pd.read_excel(xls, "genus")
ko = pd.read_excel(xls, "KO")

print(phylum.columns[:10])
print(genus.columns[:10])
print(ko.columns[:10])

Index(['phylum', 'normalized abundance (no unassigned)', 'mean abundance',
       'mean cv', 'mean subjects', 'found in datasets', 'mean abundance D01',
       'mean abundance D02', 'mean abundance D03', 'mean abundance D04'],
      dtype='object')
Index(['genus', 'normalized abundance (no unassigned)', 'mean abundance',
       'mean cv', 'mean subjects', 'found in datasets', 'mean abundance D01',
       'mean abundance D02', 'mean abundance D03', 'mean abundance D04'],
      dtype='object')
Index(['KO', 'KO name', 'pathways', 'normalized abundance (no unassigned)',
       'mean abundance', 'mean cv', 'mean subjects', 'found in datasets',
       'mean abundance D01', 'mean abundance D02'],
      dtype='object')
